In [1]:
# !pip install efficientnet_pytorch
# !pip install pretrainedmodels
# !pip install albumentations
# !pip install pandas
# !pip install sklearn

In [2]:

import torch
import albumentations

import numpy as np
import pandas as pd

import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import pretrainedmodels

from albumentations.pytorch import ToTensor
from torchvision import transforms

import albumentations as A
import random
import cv2

In [3]:
cols = ['head/neck','lower extremity','oral/genital','palms/soles','torso','unknown_anatomy','upper extremity','female','male','unknown_sex','age__0.0','age__10.0','age__15.0','age__20.0','age__25.0','age__30.0','age__35.0','age__40.0','age__45.0','age__50.0','age__55.0','age__60.0','age__65.0','age__70.0','age__75.0','age__80.0','age__85.0','age__90.0','n_images','image_size']

In [4]:
import torch

import numpy as np

from PIL import Image
from PIL import ImageFile


ImageFile.LOAD_TRUNCATED_IMAGES = True


class ClassificationLoader:
    def __init__(self, image_paths, targets, resize,augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
#         self.tabularDF = tabularDF
#         self.cols = cols

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        targets = self.targets[item]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
#         tabFeats = self.tabularDF.loc[item,cols].values.astype(float)
#         print(tabFeats)
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long)
#             "tabfeats":torch.tensor(np.array(tabFeats),dtype=torch.float)
        }


In [5]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [6]:
import torch
import numpy as np


class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.0001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(
                "EarlyStopping counter: {} out of {}".format(
                    self.counter, self.patience
                )
            )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print(
                "Validation score improved ({} --> {}). Saving model!".format(
                    self.val_score, epoch_score
                )
            )
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [7]:
class Engine:
        
    @staticmethod
    def train(
        data_loader,
        model,
        optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        use_tpu=False,
        fp16=False,
    ):

        try:
            from apex import amp
            _apex_available = True
        except ImportError:
            _apex_available = False

        if use_tpu and not _xla_available:
            raise Exception(
                "You want to use TPUs but you dont have pytorch_xla installed"
            )
        if fp16 and not _apex_available:
            raise Exception("You want to use fp16 but you dont have apex installed")
        if fp16 and use_tpu:
            raise Exception("Apex fp16 is not available when using TPUs")
        if fp16:
            accumulation_steps = 1
        losses = AverageMeter()
        predictions = []
        model.train()
        if accumulation_steps > 1:
            optimizer.zero_grad()
        tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
        for b_idx, data in enumerate(tk0):
            for key, value in data.items():
                data[key] = value.to(device)
            if accumulation_steps == 1 and b_idx == 0:
                optimizer.zero_grad()
            _, loss = model(**data)

            if not use_tpu:
                with torch.set_grad_enabled(True):
                    if fp16:
                        with amp.scale_loss(loss, optimizer) as scaled_loss:
                            scaled_loss.backward()
                    else:
                        loss.backward()
                    if (b_idx + 1) % accumulation_steps == 0:
                        optimizer.step()
                        if scheduler is not None:
                            scheduler.step()
                        if b_idx > 0:
                            optimizer.zero_grad()
            else:
                loss.backward()
                xm.optimizer_step(optimizer)
                if scheduler is not None:
                    scheduler.step()
                if b_idx > 0:
                    optimizer.zero_grad()

            losses.update(loss.item(), data_loader.batch_size)
            tk0.set_postfix(loss=losses.avg)
        return losses.avg

    @staticmethod
    def evaluate(data_loader, model, device, use_tpu=False):
        losses = AverageMeter()
        final_predictions = []
        model.eval()
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                for key, value in data.items():
                    data[key] = value.to(device)
                predictions, loss = model(**data)
                predictions = predictions.cpu()
                losses.update(loss.item(), data_loader.batch_size)
                final_predictions.append(predictions)
                tk0.set_postfix(loss=losses.avg)
        return final_predictions, losses.avg

    @staticmethod
    def predict(data_loader, model, device, use_tpu=False):
        model.eval()
        final_predictions = []
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                for key, value in data.items():
                    data[key] = value.to(device)
                predictions, _ = model(**data)
                predictions = predictions.cpu()
                final_predictions.append(predictions)
        return final_predictions

In [8]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [9]:
class Net(nn.Module):
    def __init__(self, arch):
        super(Net, self).__init__()
        self.arch = arch
        
        in_features  = arch._fc.in_features
        self.arch._fc = nn.Linear(in_features=in_features, out_features=1, bias=True)
                                
    def forward(self, image,targets):
        """
        No sigmoid in forward because we are going to use BCEWithLogitsLoss
        Which applies sigmoid for us when calculating a loss
        """
        batch_size, _, _, _ = image.shape
        x = image
        criterion = nn.BCEWithLogitsLoss()
        ### https://github.com/clovaai/CutMix-PyTorch/blob/master/train.py
        output = self.arch(x)
        loss = criterion(output, targets.view(-1,1).float())
        return output,loss

In [10]:
!ls '../input/512X512/trainwithPatientID.csv'

../input/512X512/trainwithPatientID.csv


In [11]:
trainDF_ = pd.read_csv('../input/512X512/trainwithPatientID.csv')#  -- file from kaggle competition data
trainDF_.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [12]:
df_external = pd.read_csv('../input/external/512X512/train.csv') ### file from chris external train csv
df_external.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,tfrecord,width,height
0,ISIC_0000000,-1,female,55.0,anterior torso,NV,benign,0,20,1022,767
1,ISIC_0000001,-1,female,30.0,anterior torso,NV,benign,0,14,1022,767
2,ISIC_0000002,-1,female,60.0,upper extremity,MEL,malignant,1,16,1022,767
3,ISIC_0000003,-1,male,30.0,upper extremity,NV,benign,0,0,1022,767
4,ISIC_0000004,-1,male,80.0,posterior torso,MEL,malignant,1,14,1022,767


In [13]:
cc = [i for i in trainDF_.columns if i in df_external.columns]
consol = pd.concat([trainDF_[cc],df_external[cc]],0)
consol.reset_index(drop=True,inplace=True)
consol.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [14]:
# consol.to_csv('../models/consolDF.csv',index=False)

In [15]:
import numpy as np
import random
import pandas as pd
from collections import Counter, defaultdict

def stratified_group_k_fold(X, y, groups, k, seed=None):
    """ https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation """
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in tqdm(sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])), total=len(groups_and_y_counts)):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

In [16]:
consol.tail()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
58452,ISIC_0073247,-1,female,85.0,head/neck,BCC,benign,0
58453,ISIC_0073248,-1,male,65.0,anterior torso,BKL,benign,0
58454,ISIC_0073249,-1,male,70.0,lower extremity,MEL,malignant,1
58455,ISIC_0073251,-1,female,55.0,palms/soles,NV,benign,0
58456,ISIC_0073254,-1,male,50.0,upper extremity,BKL,benign,0


In [17]:
loc = consol[consol['patient_id']==-1].index
consol.loc[loc,'patient_id'] = consol.loc[loc,'image_name'] 

In [18]:
df_folds = consol.copy(deep=True)
df_folds['source'] = 'ISIC20'
df_folds['patient_id'] = df_folds['patient_id'].fillna(df_folds['image_name'])
df_folds['sex'] = df_folds['sex'].fillna('unknown')
df_folds['anatom_site_general_challenge'] = df_folds['anatom_site_general_challenge'].fillna('unknown')
df_folds['age_approx'] = df_folds['age_approx'].fillna(round(df_folds['age_approx'].mean()))

patient_id_2_count = df_folds[['patient_id', 'image_name']].groupby('patient_id').count()['image_name'].to_dict()

df_folds = df_folds.set_index('image_name')

def get_stratify_group(row):
    stratify_group = row['sex']
#     stratify_group += f'_{row["source"]}'
    stratify_group += f'_{row["target"]}'
    patient_id_count = patient_id_2_count[row["patient_id"]]
    if patient_id_count > 80:
        stratify_group += f'_80'
    elif patient_id_count > 60:
        stratify_group += f'_60'
    elif patient_id_count > 50:
        stratify_group += f'_50'
    elif patient_id_count > 30:
        stratify_group += f'_30'
    elif patient_id_count > 20:
        stratify_group += f'_20'
    elif patient_id_count > 10:
        stratify_group += f'_10'
    else:
        stratify_group += f'_0'
    return stratify_group

df_folds['stratify_group'] = df_folds.apply(get_stratify_group, axis=1)
df_folds['stratify_group'] = df_folds['stratify_group'].astype('category').cat.codes


In [19]:
df_folds.loc[:, 'fold'] = 0

skf = stratified_group_k_fold(X=df_folds.index, y=df_folds['stratify_group'], groups=df_folds['patient_id'], k=5, seed=42)

for fold_number, (train_index, val_index) in enumerate(skf):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

df_folds.reset_index(inplace=True)
df_folds.head()

100%|██████████| 27387/27387 [01:25<00:00, 320.37it/s]


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,source,stratify_group,fold
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,ISIC20,20,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,ISIC20,2,1
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,ISIC20,0,4
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,ISIC20,2,3
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,ISIC20,1,3


In [20]:
df_folds['image_id'] = df_folds['image_name']
# df_folds.drop('image_name',inplace=True)
df_folds.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,source,stratify_group,fold,image_id
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,ISIC20,20,0,ISIC_2637011
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,ISIC20,2,1,ISIC_0015719
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,ISIC20,0,4,ISIC_0052212
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,ISIC20,2,3,ISIC_0068279
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,ISIC20,1,3,ISIC_0074268


In [21]:
df_folds.to_csv('../models/FinalFolds_combinedExternal.csv',index=False)

In [22]:
del trainDF_, consol,df_external,loc

In [23]:
class Microscope(A.ImageOnlyTransform):
    def __init__(self, p: float = 0.5, always_apply=False):
        super().__init__(always_apply, p)

    def apply(self, img, **params):
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8),
                        (img.shape[0]//2, img.shape[1]//2),
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15),
                        (0, 0, 0),
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)

        return img

In [24]:
import pretrainedmodels

def train(fold,bs,epochs,fp16,sz,arch='se_resnet152',debug=False,accumulation_steps=1):
    if sz is not None:
        sz = (sz,sz)
    else:
        sz = None
    
    _n = arch
    import os
#     training_data_path = '../input/128X128/train/'
    training_data_path = '../input/external/512X512/train/'
    df = df_folds
    device = "cuda"
    epochs = epochs
    train_bs = bs
    valid_bs = bs//2

    df_train = df[df.fold != fold].reset_index(drop=True)
    df_valid = df[df.fold == fold].reset_index(drop=True)

    
    arch = EfficientNet.from_pretrained(arch)
#     arch = pretrainedmodels.__dict__[arch](num_classes=1000, pretrained='imagenet')
    model = Net(arch=arch)  # New model for each fold
    model = model.to(device)     
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
            albumentations.CoarseDropout(),
            albumentations.RandomBrightness(0.3),
            albumentations.RandomContrast(0.3),
            albumentations.ChannelShuffle(),
            albumentations.Cutout(4,4,4),
            Microscope(),
            albumentations.ChannelDropout(p=0.1),
            albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=30),
            albumentations.Flip(p=0.5)
        ]
    )

    valid_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
        ]
    )

    if debug:
        train_images = df_train.image_id.values.tolist()[:250]
        train_images = [os.path.join(training_data_path, i + ".jpg") for i in train_images]
        train_targets = df_train.target.values[:250]

        valid_images = df_valid.image_id.values.tolist()[:250]
        valid_images = [os.path.join(training_data_path, i + ".jpg") for i in valid_images]
        valid_targets = df_valid.target.values[:250]
    else:
        train_images = df_train.image_id.values.tolist()
        train_images = [os.path.join(training_data_path, i + ".jpg") for i in train_images]
        train_targets = df_train.target.values

        valid_images = df_valid.image_id.values.tolist()
        valid_images = [os.path.join(training_data_path, i + ".jpg") for i in valid_images]
        valid_targets = df_valid.target.values
        
    train_dataset = ClassificationLoader(
        image_paths=train_images,
        targets=train_targets,
        resize=sz,
#         tabularDF = df_train,
#         cols = cols,
        augmentations=train_aug,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=train_bs, shuffle=True, num_workers=4
    )

    valid_dataset = ClassificationLoader(
        image_paths=valid_images,
        targets=valid_targets,
        resize=sz,
#         tabularDF = df_valid,
#         cols = cols,
        augmentations=valid_aug)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=valid_bs, shuffle=False, num_workers=4
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        threshold=0.001,
        mode="max"
    )
    if fp16:
        model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    es = EarlyStopping(patience=5, mode="max")
    
    for epoch in range(epochs):
        train_loss = Engine.train(train_loader, model, optimizer, device=device,fp16=fp16,accumulation_steps=accumulation_steps)
        predictions, valid_loss = Engine.evaluate(
            valid_loader, model, device=device
        )
        predictions = np.vstack((predictions)).ravel()
        auc = metrics.roc_auc_score(valid_targets, predictions)
        print(f"Epoch = {epoch}, AUC = {auc}")
        scheduler.step(auc)
        
        if sz is not None:
            ss = sz[0]
        else:
            ss = 512
            
        es(auc, model, model_path= "../models/model_arch_{}_sz_{}_fold_{}_epoch_{}_auc_{}.bin".format(_n,ss,fold,epoch,round(auc*100,2)))
        if es.early_stop:
            print("Early stopping")
            break

In [25]:
# from apex import amp, optimizers

In [26]:
e = 25
debug= False
bs = 10
accumulation_steps = 1
mtype = 'efficientnet-b6'
from apex import amp, optimizers
apx = False

# train(0,bs,e,apx,None,mtype,debug=debug,accumulation_steps=accumulation_steps)
# train(1,bs,e,apx,None,mtype,debug=debug,accumulation_steps=accumulation_steps)
# train(2,bs,e,apx,None,mtype,debug=debug,accumulation_steps=accumulation_steps)
# train(3,bs,e,apx,None,mtype,debug=debug,accumulation_steps=accumulation_steps)
train(4,bs,e,apx,None,mtype,debug=debug,accumulation_steps=accumulation_steps)

Loaded pretrained weights for efficientnet-b6


 91%|█████████ | 4229/4670 [53:38<05:35,  1.31it/s, loss=0.224]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|          | 0/4670 [00:00<?, ?it/s]

Epoch = 1, AUC = 0.9193572152375826
Validation score improved (0.898774809842628 --> 0.9193572152375826). Saving model!


  0%|          | 0/4670 [00:00<?, ?it/s]

Epoch = 2, AUC = 0.9235598742573045
Validation score improved (0.9193572152375826 --> 0.9235598742573045). Saving model!


...

Epoch = 11, AUC = 0.9445837657531202
EarlyStopping counter: 3 out of 5


  0%|          | 0/4670 [00:00<?, ?it/s]

Epoch = 12, AUC = 0.9425061411145965
EarlyStopping counter: 4 out of 5


 12%|█▏        | 563/4670 [07:08<52:04,  1.31it/s, loss=0.0829]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 2353/2353 [04:27<00:00,  8.80it/s, loss=0.154]

Epoch = 13, AUC = 0.946375470363567
EarlyStopping counter: 5 out of 5
Early stopping
